In [ ]:
#!pip install pyautogen pytesseract pillow
# !pip install --upgrade pip
# !pip install pyautogen
#!pip install -q langgraph autogen openai networkx matplotlib

## Bill Management Agent – Project Explanation

This notebook implements a **multi-agent architecture** that simulates an intelligent bill management assistant. The system consists of multiple logically separated agents, each responsible for a specific task.

---

### 1. User Proxy Agent
- Represents the end user  
- Accepts a bill image as input  
- Forwards the request to the system manager  

---

### 2. Group Manager (Orchestrator)
- Controls the workflow between agents  
- Ensures that processing happens in a sequential and structured manner  
- Routes tasks from one agent to another  

---

### 3. Bill Processing Agent
- Reads the uploaded bill image using OCR  
- Extracts text from the bill  
- Identifies expense items and amounts  
- Categorizes expenses into:
  - Groceries  
  - Dining  
  - Utilities  
  - Shopping  
  - Entertainment  
  - Others  

---

### 4. Expense Summarization Agent
- Aggregates expenses by category  
- Calculates total spending per category  
- Identifies the category with the highest expenditure  
- Detects possible high-spending trends  

---

### 5. Financial Advisor Agent (LLM-Based)
- Uses a Groq LLM (LLaMA 3) to:
  - Interpret spending data  
  - Provide cost-saving suggestions  
  - Highlig


NOTE:

This project was originally designed using the AutoGen framework for multi-agent collaboration.
However, AutoGen currently supports only OpenAI-compatible API keys.

Since I am using a Groq API key and do not have access to an OpenAI API key, AutoGen cannot be
executed without runtime warnings or failures.

To ensure error-free execution and maintain full functionality in Google Colab / Jupyter Notebook,
this implementation replaces AutoGen with a custom Python-based multi-agent orchestration system.

The design still follows the same agentic architecture:
- User Proxy Agent
- Group Manager
- Bill Processing Agent
- Expense Summarization Agent

Thus, the conceptual objectives of the assignment are fully preserved while ensuring stable execution.


In [ ]:
!pip install groq pillow pytesseract
!apt-get install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [ ]:
import os
from groq import Groq
from PIL import Image
import pytesseract
from collections import defaultdict


In [ ]:
os.environ["GROQ_API_KEY"] = " "

client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [ ]:
#CELL 4 — Create a Sample Bill Image (For Testing)
from PIL import ImageDraw

img = Image.new("RGB", (500, 350), color="white")
draw = ImageDraw.Draw(img)

bill_text = """SUPER MART
Milk 50
Bread 40
Chicken 200
Restaurant Bill 350
Electricity Bill 120
Clothes 500
Movie Ticket 250
"""

draw.text((10, 10), bill_text, fill="black")
img.save("sample_bill.jpg")

print("sample_bill.jpg created successfully")


sample_bill.jpg created successfully


In [ ]:
#AGENT IMPLEMENTATIONS (NO AUTOGEN)
#CELL 5 — User Proxy Agent
class UserProxyAgent:
    def __init__(self, name="UserProxy"):
        self.name = name

    def send_bill(self, image_path):
        print(f"[{self.name}] Sending bill image to Group Manager...")
        return {"image_path": image_path}


In [ ]:
#CELL 6 — Bill Processing Agent (OCR + Categorization)
class BillProcessingAgent:
    def __init__(self, name="BillProcessor"):
        self.name = name

    def extract_text(self, image_path):
        print(f"[{self.name}] Extracting text from bill image...")
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text

    def categorize_expenses(self, text):
        print(f"[{self.name}] Categorizing expenses...")

        categories = {
            "groceries": ["milk", "bread", "chicken", "vegetable", "fruit"],
            "dining": ["restaurant", "hotel", "cafe", "pizza"],
            "utilities": ["electricity", "water", "gas", "bill"],
            "shopping": ["clothes", "shoes", "mall"],
            "entertainment": ["movie", "ticket", "netflix"]
        }

        categorized = defaultdict(list)

        for line in text.lower().split("\n"):
            words = line.split()
            if len(words) >= 2:
                item = " ".join(words[:-1])
                try:
                    amount = float(words[-1])
                except:
                    continue

                matched = False
                for category, keywords in categories.items():
                    if any(k in item for k in keywords):
                        categorized[category].append((item, amount))
                        matched = True
                        break

                if not matched:
                    categorized["others"].append((item, amount))

        return categorized


In [ ]:
#CELL 7 — Expense Summarization Agent
class ExpenseSummarizationAgent:
    def __init__(self, name="ExpenseSummarizer"):
        self.name = name

    def summarize(self, categorized_expenses):
        print(f"[{self.name}] Summarizing expenses...")

        summary = {}
        for category, items in categorized_expenses.items():
            total = sum(amount for _, amount in items)
            summary[category] = total

        highest_category = max(summary, key=summary.get)

        insights = {
            "total_by_category": summary,
            "highest_spending_category": highest_category,
            "highest_amount": summary[highest_category]
        }

        return insights


In [ ]:
#CELL 8 — Group Manager (Agent Orchestration)
class GroupManager:
    def __init__(self):
        self.bill_agent = BillProcessingAgent()
        self.summary_agent = ExpenseSummarizationAgent()

    def handle_request(self, user_message):
        image_path = user_message["image_path"]

        text = self.bill_agent.extract_text(image_path)
        categorized = self.bill_agent.categorize_expenses(text)
        insights = self.summary_agent.summarize(categorized)

        return categorized, insights


In [ ]:
#LLM-BASED FINAL REPORT (USING GROQ)
#CELL 9 — Financial Advisor Agent (Groq LLM)
class FinancialAdvisorAgent:
    def __init__(self, client):
        self.client = client

    def generate_report(self, insights):
        prompt = f"""
You are a financial advisor.

Here is the user's spending summary:
{insights}

Tasks:
1. Identify the highest spending category.
2. Provide 2 cost-saving suggestions.
3. Mention if any spending appears unusually high.

Respond in a professional tone.
"""

        response = self.client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2
        )

        return response.choices[0].message.content


In [ ]:
#CELL 10 — Run the Complete Workflow
# Initialize agents
user_agent = UserProxyAgent()
group_manager = GroupManager()
advisor_agent = FinancialAdvisorAgent(client)

# Step 1: User sends bill image
user_input = user_agent.send_bill("sample_bill.jpg")

# Step 2: Group Manager orchestrates processing
categorized_expenses, insights = group_manager.handle_request(user_input)

print("\n--- Categorized Expenses ---")
for category, items in categorized_expenses.items():
    print(category.upper())
    for item, amount in items:
        print(f"  {item} : {amount}")

print("\n--- Summary ---")
for cat, total in insights["total_by_category"].items():
    print(f"{cat} : {total}")

print("\nHighest Spending Category:", insights["highest_spending_category"])

# Step 3: Generate LLM-based financial report
final_report = advisor_agent.generate_report(insights)

print("\n--- FINAL FINANCIAL REPORT ---")
print(final_report)


[UserProxy] Sending bill image to Group Manager...
[BillProcessor] Extracting text from bill image...
[BillProcessor] Categorizing expenses...
[ExpenseSummarizer] Summarizing expenses...

--- Categorized Expenses ---
GROCERIES
  bread : 40.0
  chicken : 200.0
ENTERTAINMENT
  movie ticket : 250.0

--- Summary ---
groceries : 240.0
entertainment : 250.0

Highest Spending Category: entertainment

--- FINAL FINANCIAL REPORT ---
Based on the provided spending summary, I have analyzed the data and identified the following key points:

**Highest Spending Category:**
The highest spending category is 'entertainment', with a total expenditure of $250.00.

**Cost-Saving Suggestions:**
To optimize your spending, I recommend the following two cost-saving suggestions:

1. **Budget Allocation:** Consider allocating a specific budget for entertainment expenses, such as dining out or movie nights. This will help you stay within your means and avoid overspending. You can also explore alternative, more a